In [2]:
import sagemaker
from sagemaker import get_execution_role
import bimport tensorflow as tf
from tensorflow.keras.models import load_model

# Load the Keras .h5 model
model = load_model('pneumonia_detection_model.h5')

# Export to TensorFlow's SavedModel format
model.save('pneumonia_detection_saved_model/1')
oto3


In [3]:
print("hello")

hello


In [4]:
role = get_execution_role()
bucket = 'pneumonia-detection-model-bucket'  
model_path = 's3://pneumonia-detection-model-bucket/pneumonia_detection_model.h5'
print("ok")

ok


In [6]:
# import tensorflow as tf
# from tensorflow.keras.models import load_model
# model = load_model('pneumonia_detection_model.h5')

# #export to tensorflow's SavedModel format
# model.save('pneumonia_detection_saved_model/1')

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization

# Reconstruct the model architecture
def create_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), strides=1, padding='same', activation='relu', input_shape=(150, 150, 1)))
    model.add(BatchNormalization())
    model.add(MaxPool2D((2, 2), strides=2, padding='same'))

    model.add(Conv2D(64, (3, 3), strides=1, padding='same', activation='relu'))
    model.add(Dropout(0.1))
    model.add(BatchNormalization())
    model.add(MaxPool2D((2, 2), strides=2, padding='same'))

    model.add(Conv2D(64, (3, 3), strides=1, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D((2, 2), strides=2, padding='same'))

    model.add(Conv2D(128, (3, 3), strides=1, padding='same', activation='relu'))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    model.add(MaxPool2D((2, 2), strides=2, padding='same'))

    model.add(Conv2D(256, (3, 3), strides=1, padding='same', activation='relu'))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    model.add(MaxPool2D((2, 2), strides=2, padding='same'))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer="rmsprop", loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Load weights from the existing model
try:
    model = create_model()
    model.load_weights('pneumonia_detection_model.h5')
    print("Model architecture loaded and weights applied successfully.")
except Exception as e:
    print("Error loading weights:", e)

# Save the model in TensorFlow format
if model is not None:
    try:
        model.save('pneumonia_detection_saved_model/1', save_format='tf')
        print("Model saved in TensorFlow format.")
    except Exception as e:
        print("Error saving model:", e)





Model architecture loaded and weights applied successfully.
INFO:tensorflow:Assets written to: pneumonia_detection_saved_model/1/assets


INFO:tensorflow:Assets written to: pneumonia_detection_saved_model/1/assets


Model saved in TensorFlow format.


In [7]:
loaded_model = tf.keras.models.load_model('pneumonia_detection_saved_model/1')
print("Model loaded successfully.")


Model loaded successfully.


In [14]:
import numpy as np

test_image = np.random.rand(1, 150, 150, 1)  # Dummy input; replace with actual image data

prediction = loaded_model.predict(test_image)
print(f"Prediction: {prediction}")


1/1 [==============================] - 0s 29ms/step
Prediction: [[0.18780714]]
hello


In [17]:
# import boto3

# sagemaker = boto3.client('sagemaker')

# model_name = 'pneumonia-detection-model'
# s3_model_path = 's3://pneumonia-detection-bucket/pneumonia_detection_saved_model'

# response = sagemaker.create_model(
#     ModelName=model_name,
#     PrimaryContainer={
#         'Image': '763104351884.dkr.ecr.eu-north-1.amazonaws.com/tensorflow-inference:latest',
#         'ModelDataUrl': s3_model_path,
#         'Environment': {
#             'SAGEMAKER_PROGRAM': 's3://pneumonia-detection-bucket/inference.py',
#             'SAGEMAKER_SUBMIT_DIRECTORY': s3_model_path
#         }
#     },
#     ExecutionRoleArn='arn:aws:iam::205930617574:role/PneumoniaDetection-SageMakerRole'
# )

# import boto3

# # Initialize SageMaker client
# sagemaker = boto3.client('sagemaker')

# model_name = 'pneumonia-detection-model'
# s3_model_path = 's3://pneumonia-detection-bucket/pneumonia_detection_saved_model'

# response = sagemaker.create_model(
#     ModelName=model_name,
#     PrimaryContainer={
#         'Image': '763104351884.dkr.ecr.eu-north-1.amazonaws.com/tensorflow-inference:2.16.0-gpu',  # Updated image
#         'ModelDataUrl': s3_model_path,
#         'Environment': {
#             'SAGEMAKER_PROGRAM': 'inference.py',  # This should just be the filename, assuming it's in S3
#             'SAGEMAKER_SUBMIT_DIRECTORY': s3_model_path
#         }
#     },
#     ExecutionRoleArn='arn:aws:iam::205930617574:role/PneumoniaDetection-SageMakerRole'
# )


import sagemaker
import boto3

# Specify framework and version
framework = 'tensorflow'
framework_version = '2.7.0'
instance_type = 'ml.m5.large'

# Retrieve the image URI for TensorFlow
image_uri = sagemaker.image_uris.retrieve(
    framework=framework,
    region='eu-north-1',
    version=framework_version,
    instance_type=instance_type,
    image_scope='inference'  # Use 'training' if you're training the model
)

print(f"Using image URI: {image_uri}")



sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
Using image URI: 763104351884.dkr.ecr.eu-north-1.amazonaws.com/tensorflow-inference:2.7.0-cpu


In [22]:
# Create the model using the retrieved image URI
sagemaker_client = boto3.client('sagemaker')

model_name = 'pneumonia-detection-model'
s3_model_path = 's3://pneumonia-detection-model-bucket/pneumonia_detection_saved_model/pneumonia_detection_saved_model.tar.gz'

response = sagemaker_client.create_model(
    ModelName=model_name,
    PrimaryContainer={
        'Image': image_uri,  # Use the dynamically retrieved image URI
        'ModelDataUrl': s3_model_path,
        'Environment': {
            'SAGEMAKER_PROGRAM': 'inference.py',  # Your inference script
            'SAGEMAKER_SUBMIT_DIRECTORY': s3_model_path
        }
    },
    ExecutionRoleArn='arn:aws:iam::205930617574:role/PneumoniaDetection-SageMakerRole'
)

print("Model created:", response['ModelArn'])



Model created: arn:aws:sagemaker:eu-north-1:205930617574:model/pneumonia-detection-model


In [20]:
!aws s3 cp pneumonia_detection_saved_model.tar.gz s3://pneumonia-detection-model-bucket/pneumonia_detection_saved_model/

upload failed: ./pneumonia_detection_saved_model.tar.gz to s3://pneumonia-detection-model-bucket/pneumonia_detection_saved_model/pneumonia_detection_saved_model.tar.gz An error occurred (AccessDenied) when calling the PutObject operation: User: arn:aws:sts::205930617574:assumed-role/PneumoniaDetection-SageMakerRole/SageMaker is not authorized to perform: s3:PutObject on resource: "arn:aws:s3:::pneumonia-detection-model-bucket/pneumonia_detection_saved_model/pneumonia_detection_saved_model.tar.gz" because no identity-based policy allows the s3:PutObject action


In [25]:
import boto3

sagemaker_client = boto3.client('sagemaker')

response = sagemaker_client.create_endpoint_config(
    EndpointConfigName='pneumonia-detection-config',
    ProductionVariants=[
        {
            'VariantName': 'AllTraffic',
            'ModelName': 'pneumonia-detection-model',
            'InstanceType': 'ml.m5.large',  # Choose an appropriate instance type
            'InitialInstanceCount': 1,
            'InitialVariantWeight': 1
        }
    ]
)
print("Endpoint configuration created:", response['EndpointConfigArn'])


Endpoint configuration created: arn:aws:sagemaker:eu-north-1:205930617574:endpoint-config/pneumonia-detection-config


In [26]:
response = sagemaker_client.create_endpoint(
    EndpointName='pneumonia-detection-endpoint',
    EndpointConfigName='pneumonia-detection-config'
)
print("Endpoint creation initiated:", response['EndpointArn'])


Endpoint creation initiated: arn:aws:sagemaker:eu-north-1:205930617574:endpoint/pneumonia-detection-endpoint


In [27]:
import time

while True:
    status = sagemaker_client.describe_endpoint(EndpointName='pneumonia-detection-endpoint')['EndpointStatus']
    print("Endpoint status:", status)
    if status in ['Creating', 'Updating']:
        time.sleep(30)  # Wait before checking again
    else:
        break

print("Final endpoint status:", status)


Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: InService
Final endpoint status: InService


In [35]:
import boto3
import json
import numpy as np
from PIL import Image

# Load the image
image_path = 'sample_normal.jpeg'  # Update with the uploaded image path
image = Image.open(image_path)

# Convert to grayscale if needed
if image.mode != 'L':  # 'L' is for grayscale
    image = image.convert('L')

image = image.resize((150, 150))  # Resize to match your model input size
image_array = np.array(image) / 255.0  # Normalize if needed

# Ensure the shape is (1, 150, 150, 1)
image_array = image_array.reshape((1, 150, 150, 1)).tolist()  # Reshape for your model

# Create SageMaker runtime client
runtime_client = boto3.client('sagemaker-runtime', region_name='eu-north-1')

# Invoke the endpoint
response = runtime_client.invoke_endpoint(
    EndpointName='pneumonia-detection-endpoint',
    ContentType='application/json',
    Body=json.dumps({'instances': [{'data': image_array}]})
)

# Read and print the response
print("Response Status Code:", response['ResponseMetadata']['HTTPStatusCode'])
print("Response Body:", response['Body'].read().decode())


ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (400) from primary with message "{
    "error": "Failed to process element: 0 key: data of 'instances' list. Error: INVALID_ARGUMENT: JSON object: does not have named input: data"
}". See https://eu-north-1.console.aws.amazon.com/cloudwatch/home?region=eu-north-1#logEventViewer:group=/aws/sagemaker/Endpoints/pneumonia-detection-endpoint in account 205930617574 for more information.

In [36]:
import numpy as np
from PIL import Image

# Load the model (assuming it's already loaded)
# loaded_model = load_model('your_model_path')

# Load the image
image_path = 'sample_normal.jpeg'  # Update with the actual image path
image = Image.open(image_path)

# Convert to grayscale if needed
if image.mode != 'L':  # 'L' is for grayscale
    image = image.convert('L')

# Resize the image to match model input size
image = image.resize((150, 150))

# Convert to numpy array and normalize
image_array = np.array(image) / 255.0  # Normalize if needed

# Ensure the shape is (1, 150, 150, 1)
image_array = image_array.reshape((1, 150, 150, 1))

# Make the prediction
prediction = loaded_model.predict(image_array)
print(f"Prediction: {prediction}")


1/1 [==============================] - 0s 32ms/step
Prediction: [[0.8990011]]


In [38]:
import boto3

sagemaker_client = boto3.client('sagemaker', region_name='eu-north-1')
model_data = 's3://pneumonia-detection-model-bucket/pneumonia_detection_saved_model/pneumonia_detection_saved_model.tar.gz'

# creating sagemaker model
model_name = 'pneumonia-detection-model-v2'

create_model_response = sagemaker_client.create_model(
    ModelName=model_name,
    PrimaryContainer={
        'Image': '763104351884.dkr.ecr.eu-north-1.amazonaws.com/tensorflow-inference:2.7.0-cpu',  # tf container
        'ModelDataUrl': model_data,
        'Environment': {
            'SAGEMAKER_REGION': 'eu-north-1'
        }
    },
    ExecutionRoleArn='arn:aws:iam::205930617574:role/PneumoniaDetection-SageMakerRole'  
)

print(f"Model ARN: {create_model_response['ModelArn']}")

# creating endpoint config
endpoint_config_name = 'pneumonia-detection-endpoint-config-v2'

create_endpoint_config_response = sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[{
        'VariantName': 'AllTraffic',
        'ModelName': model_name,
        'InstanceType': 'ml.m5.large',  
        'InitialInstanceCount': 1,
        'InitialVariantWeight': 1
    }]
)

print(f"Endpoint Config ARN: {create_endpoint_config_response['EndpointConfigArn']}")

# deploy endpoint
endpoint_name = 'pneumonia-detection-endpoint-v2'

create_endpoint_response = sagemaker_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name
)

print(f"Endpoint ARN: {create_endpoint_response['EndpointArn']}")


Model ARN: arn:aws:sagemaker:eu-north-1:205930617574:model/pneumonia-detection-model-v2
Endpoint Config ARN: arn:aws:sagemaker:eu-north-1:205930617574:endpoint-config/pneumonia-detection-endpoint-config-v2
Endpoint ARN: arn:aws:sagemaker:eu-north-1:205930617574:endpoint/pneumonia-detection-endpoint-v2
